# PRIMERA FUENTE DE DATOS: World Bank

In [ ]:
# - requests: para obtener datos de Internet.
# - pandas: para organizar los datos en tablas.
# - json: para trabajar con información en formato JSON.
import requests
import pandas as pd
import json

# URL de la API del Banco Mundial donde están los datos de anemia infantil.
base_url = "http://api.worldbank.org/v2/country/ALL/indicator/SH.ANM.CHLD.ZS"

# Indicamos a la API que queremos los datos en formato JSON.
params = {
    "format": "json"
}

# Creamos una lista vacía donde guardaremos los datos.
all_data = []

# Enviamos la solicitud a la API.
response = requests.get(base_url, params=params)

# Verificamos si la solicitud fue exitosa (código 200 significa éxito).
if response.status_code == 200:
    # Convertimos la respuesta de la API en un formato que Python pueda entender (JSON).
    data = response.json()
    
    # Revisamos si hay datos útiles en la respuesta.
    if len(data) > 1:  # Los datos que necesitamos están en la segunda parte de la respuesta.
        all_data = data[1]  # Guardamos esos datos en nuestra lista.
    else:
        print("No se encontraron datos en la respuesta.")
else:
    # Si ocurre un error, mostramos el código de error.
    print("Error al obtener los datos:", response.status_code)

# Ahora, organizamos los datos en una tabla usando pandas.
df_worldbank = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV (una hoja de cálculo).
output_file = "world_bank_anemia.csv"
df_worldbank.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos el archivo sin incluir índices.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que se creó el archivo.

# SEGUNDA FUENTE: Global Health Observatory

In [ ]:
import requests
import pandas as pd
import json

# URL de la API para datos de nutrición y anemia infantil.
base_url = "https://ghoapi.azureedge.net/api/NUTRITION_ANAEMIA_CHILDREN_NUM"

# Configuramos los parámetros para obtener los datos en partes (paginación):
# - "$top": cuántos registros obtener por solicitud.
# - "$skip": cuántos registros saltar para la siguiente solicitud.
params = {
    "$top": 1000,  # Pedimos 1000 registros por solicitud.
    "$skip": 0     # Empezamos desde el inicio.
}

# Lista vacía para guardar todos los datos.
all_data = []

# Usamos un ciclo para seguir pidiendo datos hasta que no queden más.
while True:
    # Hacemos una solicitud a la API con los parámetros actuales.
    response = requests.get(base_url, params=params)
    
    # Si la solicitud fue exitosa:
    if response.status_code == 200:
        # Convertimos la respuesta en un formato JSON y agregamos los datos a nuestra lista.
        data = response.json()
        all_data.extend(data["value"])  # Extendemos la lista con los nuevos datos.
        
        # Si la cantidad de datos obtenidos es menor que "$top", significa que no hay más datos.
        if len(data["value"]) < params["$top"]:
            break  # Terminamos el ciclo.
        
        # Si hay más datos, incrementamos "$skip" para pedir la siguiente página.
        params["$skip"] += params["$top"]
    else:
        # Si ocurre un error, mostramos el código de error y terminamos el ciclo.
        print("Error al obtener los datos:", response.status_code)
        break

# Organizamos los datos en una tabla con pandas.
df_anemia = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV.
output_file = "nutrition_anemia_children.csv"
df_anemia.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos el archivo.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que se creó el archivo.

# TERCERA FUENTE: DHS Program

### ENCUESTA 1: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON CUALQUIER TIPO DE ANEMIA

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de DHS para obtener datos de nutrición (anemia en este caso).
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_ANY"

# Parámetros que se enviarán a la API:
# - "perpage": número máximo de registros por solicitud (aquí pedimos 1000).
# - "page": indica el número de la página que estamos solicitando (empezamos en la 1).
params = {
    "perpage": 1000,  # Máximo de registros por página.
    "page": 1         # Comenzamos desde la página 1.
}

# Creamos una lista vacía para almacenar todos los datos obtenidos de la API.
all_data = []

# Usamos un ciclo para descargar todos los datos disponibles en la API.
while True:
    # Realizamos una solicitud a la API con los parámetros actuales.
    response = requests.get(base_url, params=params)
    
    # Verificamos si la solicitud fue exitosa (código de respuesta 200).
    if response.status_code == 200:
        # Convertimos la respuesta de la API a formato JSON (fácil de manejar en Python).
        data = response.json()
        
        # Extraemos los datos dentro de la clave "Data" y los agregamos a nuestra lista.
        # Usamos "get" para evitar errores si la clave no existe.
        all_data.extend(data.get("Data", []))
        
        # Verificamos si hemos llegado al final de los datos:
        # Si la cantidad de datos obtenidos es menor que el límite "perpage",
        # significa que no hay más páginas que consultar.
        if len(data.get("Data", [])) < params["perpage"]:
            break  # Salimos del ciclo porque ya no hay más datos.
        
        # Si aún hay más datos, incrementamos el número de página para la siguiente solicitud.
        params["page"] += 1
    else:
        # Si ocurre un error (código distinto de 200), mostramos el código de error y terminamos.
        print("Error al obtener los datos:", response.status_code)
        break

# Una vez descargados todos los datos, los organizamos en una tabla con Pandas.
df_any = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV llamado "df_any_anemia.csv".
output_file = "df_any_anemia.csv"
df_any.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos sin incluir índices y en UTF-8.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que el archivo se creó.

### ENCUESTA 2: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA LEVE

In [ ]:
import requests
import pandas as pd
import json
# URL de la API del DHS para obtener datos de anemia infantil leve.
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_MLD"

# Parámetros que enviamos a la API:
# - "perpage": cantidad máxima de datos que queremos recibir por solicitud (1000 aquí).
# - "page": indica el número de página que solicitamos (iniciamos desde la página 1).
params = {
    "perpage": 1000,  # Pedimos hasta 1000 registros por página.
    "page": 1         # Empezamos desde la primera página.
}

# Creamos una lista vacía para guardar todos los datos descargados de la API.
all_data = []

# Usamos un ciclo para realizar solicitudes a la API hasta que descarguemos todos los datos.
while True:
    # Realizamos la solicitud a la API con los parámetros configurados.
    response = requests.get(base_url, params=params)
    
    # Verificamos si la solicitud fue exitosa (código 200 indica éxito).
    if response.status_code == 200:
        # Convertimos la respuesta en formato JSON (fácil de manejar en Python).
        data = response.json()
        
        # Agregamos los datos obtenidos (clave "Data") a nuestra lista.
        # Usamos "get" para evitar errores si la clave no existe.
        all_data.extend(data.get("Data", []))
        
        # Verificamos si ya no hay más datos disponibles:
        # Si el número de datos recibidos es menor al límite "perpage", hemos llegado al final.
        if len(data.get("Data", [])) < params["perpage"]:
            break  # Salimos del ciclo porque no hay más datos.
        
        # Si aún hay más datos, pasamos a la siguiente página incrementando el número de página.
        params["page"] += 1
    else:
        # Si ocurre un error (respuesta distinta de 200), mostramos un mensaje con el código de error.
        print("Error al obtener los datos:", response.status_code)
        break

# Cuando terminamos de descargar los datos, los organizamos en una tabla usando pandas.
df_mld = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV llamado "df_mld_anemia.csv".
output_file = "df_mld_anemia.csv"
df_mld.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos sin índices y en formato UTF-8.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que el archivo fue creado.

Archivo CSV creado exitosamente: df_mld_anemia.csv


### ENCUESTA 3: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA MODERADA

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de DHS para obtener datos de anemia infantil moderada.
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_MOD"

# Parámetros que enviamos a la API:
# - "perpage": define el número máximo de registros a recibir por solicitud (en este caso, 1000).
# - "page": indica el número de página que se solicita (empezamos en la página 1).
params = {
    "perpage": 1000,  # Pedimos hasta 1000 registros por página.
    "page": 1         # Comenzamos desde la primera página.
}

# Lista vacía para almacenar todos los datos descargados de la API.
all_data = []

# Ciclo para realizar solicitudes repetidas hasta que se descarguen todos los datos disponibles.
while True:
    # Realizamos una solicitud GET a la API utilizando la URL base y los parámetros definidos.
    response = requests.get(base_url, params=params)
    
    # Verificamos si la solicitud fue exitosa (código de estado 200).
    if response.status_code == 200:
        # Convertimos la respuesta en formato JSON (fácil de manipular en Python).
        data = response.json()
        
        # Extraemos los datos de la clave "Data" y los agregamos a la lista "all_data".
        # Usamos "get" para evitar errores si la clave no está presente.
        all_data.extend(data.get("Data", []))
        
        # Verificamos si hemos llegado al final de los datos disponibles:
        # Si el número de registros recibidos es menor que "perpage", significa que no hay más páginas.
        if len(data.get("Data", [])) < params["perpage"]:
            break  # Salimos del ciclo porque ya no hay más datos.
        
        # Si todavía hay más datos, incrementamos el número de página para la siguiente solicitud.
        params["page"] += 1
    else:
        # Si ocurre un error (código diferente de 200), mostramos el código de error y detenemos el proceso.
        print("Error al obtener los datos:", response.status_code)
        break

# Organizamos los datos descargados en una tabla (DataFrame) usando pandas.
df_mod = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV llamado "df_mod_anemia.csv".
output_file = "df_mod_anemia.csv"
df_mod.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos sin incluir índices y en formato UTF-8.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que el archivo fue creado correctamente.

Archivo CSV creado exitosamente: df_mod_anemia.csv


### ENCUESTA 4: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA GRAVE

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de DHS (Nutrición - Anemia infantil moderada)
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_SEV"

# Definir los parámetros para la solicitud (si soporta paginación)
params = {
    "perpage": 1000,  # Número de registros por página (se solicita un máximo de 1000 registros por solicitud)
    "page": 1         # Página inicial, comenzamos desde la primera página de resultados
}

# Lista para almacenar todos los datos que se vayan obteniendo
all_data = []

# Hacer varias solicitudes hasta obtener todos los datos (paginación)
while True:
    # Realizar la solicitud a la API utilizando los parámetros definidos
    response = requests.get(base_url, params=params)
    
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta en formato JSON para manejar los datos
        data = response.json()
        
        # Extraer la lista de datos ("Data") de la respuesta JSON y agregarla a all_data
        all_data.extend(data.get("Data", []))
        
        # Verificar si hemos recibido menos registros de los solicitados, lo que indica que hemos llegado al final
        if len(data.get("Data", [])) < params["perpage"]:
            break
        
        # Incrementar el número de página para la siguiente solicitud
        params["page"] += 1
    else:
        # En caso de error, imprimir el código de estado de la respuesta
        print("Error al obtener los datos:", response.status_code)
        break

# Convertir los datos obtenidos (all_data) a un DataFrame de Pandas para facilitar su análisis
df_sev = pd.json_normalize(all_data)

# Guardar el DataFrame en un archivo CSV
output_file = "df_sev_anemia.csv"
df_sev.to_csv(output_file, index=False, encoding="utf-8")

# Imprimir mensaje de éxito indicando que el archivo CSV fue creado correctamente
print(f"Archivo CSV creado exitosamente: {output_file}")


Archivo CSV creado exitosamente: df_sev_anemia.csv


# CUARTA FUENTE: KAGGLE

In [ ]:
pip install kaggle

In [ ]:
import json
import os

# Ruta al archivo 'kaggle.json' que contiene las credenciales de acceso a la API de Kaggle
kaggle_file_path = 'kaggle.json'

# Leer el archivo JSON que contiene las credenciales
with open(kaggle_file_path, 'r') as file:
    kaggle_api = json.load(file)

# Extraer el 'username' y la 'key' de las credenciales de Kaggle
kaggle_username = kaggle_api['username']  # Obtiene el nombre de usuario de Kaggle
kaggle_key = kaggle_api['key']  # Obtiene la clave de la API de Kaggle

# Configurar las variables de entorno con las credenciales para acceder a la API de Kaggle
os.environ['KAGGLE_USERNAME'] = kaggle_username  # Establece la variable de entorno para el nombre de usuario
os.environ['KAGGLE_KEY'] = kaggle_key  # Establece la variable de entorno para la clave de la API

# Imprimir un mensaje de confirmación indicando que las credenciales han sido configuradas correctamente
print("Credenciales de Kaggle configuradas correctamente.")

In [ ]:
import subprocess

# Ejecutar el comando para listar datasets relacionados con "child anemia" en Kaggle y capturar la salida
result = subprocess.run(
    ["kaggle", "datasets", "list", "-s", "child anemia"],  # El comando Kaggle para buscar datasets
    capture_output=True,  # Captura tanto la salida estándar como los errores
    text=True  # Decirle a subprocess que el resultado debe ser tratado como texto (en lugar de bytes)
)

# Imprimir la salida del comando ejecutado
print(result.stdout)  # Muestra el resultado del comando (en este caso, la lista de datasets)



### DATA 1: Factores que afectan el nivel de anemia en los niños (Estudio en Nigeria)


In [ ]:
import subprocess

# Nombre del dataset a descargar desde Kaggle
dataset_name = "adeolaadesina/factors-affecting-children-anemia-level"

# Ejecutar el comando para descargar y descomprimir el dataset
subprocess.run(["kaggle", "datasets", "download", "-d", dataset_name, "--unzip"])


In [ ]:
import pandas as pd

# Verifica el nombre del archivo descargado y ajústalo aquí
csv_file = "children anemia.csv"  # El nombre del archivo CSV descargado

# Leer el archivo CSV en un DataFrame de Pandas
df_nigeria = pd.read_csv(csv_file)

# Confirmar que los datos se han leído correctamente
print("Datos extraídos exitosamente de Kaggle (Caso Nigeria)")


### DATA 2: Encuesta Nacional de Familia y Salud (Estudio en ¿India?)

In [ ]:
import subprocess

# Nombre del dataset que quieres descargar desde Kaggle
dataset_name = "ravisinghiitbhu/nfhs5"  # Identificador del dataset

# Ejecutar el comando para descargar y descomprimir el dataset
subprocess.run(["kaggle", "datasets", "download", "-d", dataset_name, "--unzip"])


In [ ]:
import pandas as pd

## Verifica el nombre del archivo descargado y ajústalo aquí
csv_file = "Final.csv"  # Nombre del archivo CSV descargado desde Kaggle

# Leer el archivo CSV en un DataFrame de Pandas
df_india = pd.read_csv(csv_file)

# Confirmar que los datos se han leído correctamente
print("Datos extraídos exitosamente de Kaggle (Caso India)")


# **Módulo Análisis de Datos**

## API: World Bank Prevalencia de anemia infantil (% de anemia infantil entre los 6-59 meses)

## 1. Limpieza de Datos
### 1.1 Primera revisión del archivo para observar sus columnas y contenido de filas

In [ ]:
import pandas as pd

# Cargar el archivo proporcionado por el usuario
file_path = 'world_bank_anemia.csv'
data = pd.read_csv(file_path)

# Mostrar una vista previa de los datos para analizar la estructura
data.head(), data.info()

# Obtener el nombre de las columnas
columnas = data.columns.tolist()

print("Nombres de las columnas:", columnas)

### 1.2 Eliminamos las columnas innecesarias y filas sin valores de anemia, también redondeamos valores de anemia

In [ ]:
# Cargar el archivo CSV original
archivo_original = "world_bank_anemia.csv"  # Nombre de tu archivo
df = pd.read_csv(archivo_original)

# Especificar las columnas a eliminar
columnas_a_eliminar = ["unit", "obs_status", "decimal", "indicator.id","countryiso3code","country.id", "indicator.value"]  # Cambia según lo que quieras eliminar

# Redondear los valores de la columna "value" a 1 decimal
df['value'] = df['value'].round(1)

# Eliminar las columnas especificadas
df_filtrado = df.drop(columns=columnas_a_eliminar)

# Eliminar filas donde la columna 'value' esté vacía
world_bank_anemia_limpio = df_filtrado.dropna(subset=['value'])

## 2. Filtrar
### 2.1 Filtrar CSV: Separación por países

In [ ]:
# Cargar el archivo CSV original
archivo_csv = "world_bank_anemia_limpio.csv"  # Reemplaza con la ruta a tu archivo
df = pd.read_csv(archivo_csv)

# Filtrar las filas desde "Afghanistan" hasta "Zimbabwe" en la última columna
ultima_columna = world_bank_anemia_limpio.columns[-1]  # Nombre de la última columna
filtro_paises = world_bank_anemia_limpio[ultima_columna].str.strip().isin(["Afghanistan", "Zimbabwe"])  # Filtrar inicio y fin

# Obtener los índices del rango
inicio = world_bank_anemia_limpio[filtro_paises].index.min()  # Índice de "Afghanistan"
fin = world_bank_anemia_limpio[filtro_paises].index.max()  # Índice de "Zimbabwe"

# Seleccionar los datos dentro de este rango
df_paises = world_bank_anemia_limpio.iloc[inicio:fin + 1]  # Incluye ambas filas

### 2.1.1 Mejoramos el nombre de las columnas:

In [ ]:
# Cambiar los nombres de las columnas
df_paises = df_paises.rename(columns={
    'date': 'year',
    'value': 'prevalencia (%)',
    'country.value': 'pais'
})

# Guardar el nuevo archivo CSV con los nombres de columnas cambiados
archivo_csv_nuevo = "world_bank_anemia_paises_listo.csv"
df.to_csv(archivo_csv_nuevo, index=False)

print(f"Archivo con las columnas renombradas guardado como {archivo_csv_nuevo}")

### 2.2 Agrupar por continentes

In [ ]:
# Cargar el archivo CSV
file_path = 'world_bank_anemia_paises_listo.csv'
data = pd.read_csv(file_path)

# Diccionario para mapear países a continentes
country_to_continent = {
    "Afghanistan": "Asia",
    "Albania": "Europe",
    "Algeria": "Africa",
    "Andorra": "Europe",
    "Angola": "Africa",
    "Antigua and Barbuda": "North America",
    "Argentina": "South America",
    "Armenia": "Asia",
    "Australia": "Oceania",
    "Austria": "Europe",
    "Azerbaijan": "Asia",
    "Bahamas": "North America",
    "Bahamas, The": "North America",
    "Bahrain": "Asia",
    "Bangladesh": "Asia",
    "Barbados": "North America",
    "Belarus": "Europe",
    "Belgium": "Europe",
    "Belize": "North America",
    "Benin": "Africa",
    "Bhutan": "Asia",
    "Bolivia": "South America",
    "Bosnia and Herzegovina": "Europe",
    "Botswana": "Africa",
    "Brazil": "South America",
    "Brunei Darussalam": "Asia",
    "Bulgaria": "Europe",
    "Burkina Faso": "Africa",
    "Burundi": "Africa",
    "Cabo Verde": "Africa",
    "Cambodia": "Asia",
    "Cameroon": "Africa",
    "Canada": "North America",
    "Central African Republic": "Africa",
    "Chad": "Africa",
    "Chile": "South America",
    "China": "Asia",
    "Colombia": "South America",
    "Comoros": "Africa",
    "Congo, Dem. Rep.": "Africa",
    "Congo, Rep.": "Africa",
    "Costa Rica": "North America",
    "Cote d'Ivoire": "Africa",
    "Croatia": "Europe",
    "Cuba": "North America",
    "Cyprus": "Asia",
    "Czech Republic": "Europe",
    "Czechia": "Europe",
    "Denmark": "Europe",
    "Djibouti": "Africa",
    "Dominica": "North America",
    "Dominican Republic": "North America",
    "Ecuador": "South America",
    "Egypt": "Africa",
    "Egypt, Arab Rep.": "Africa",
    "El Salvador": "North America",
    "Equatorial Guinea": "Africa",
    "Eritrea": "Africa",
    "Estonia": "Europe",
    "Eswatini": "Africa",
    "Ethiopia": "Africa",
    "Fiji": "Oceania",
    "Finland": "Europe",
    "France": "Europe",
    "Gabon": "Africa",
    "Gambia": "Africa",
    "Gambia, The": "Africa",
    "Georgia": "Asia",
    "Germany": "Europe",
    "Ghana": "Africa",
    "Greece": "Europe",
    "Grenada": "North America",
    "Guatemala": "North America",
    "Guinea": "Africa",
    "Guinea-Bissau": "Africa",
    "Guyana": "South America",
    "Haiti": "North America",
    "Honduras": "North America",
    "Hungary": "Europe",
    "Iceland": "Europe",
    "India": "Asia",
    "Indonesia": "Asia",
    "Iran": "Asia",
    "Iran, Islamic Rep.": "Asia",
    "Iraq": "Asia",
    "Ireland": "Europe",
    "Israel": "Asia",
    "Italy": "Europe",
    "Jamaica": "North America",
    "Japan": "Asia",
    "Jordan": "Asia",
    "Kazakhstan": "Asia",
    "Kenya": "Africa",
    "Kiribati": "Oceania",
    "Korea, Dem. People's Rep.": "Asia",
    "Korea, Dem. Rep.": "Asia",
    "Korea, Rep.": "Asia",
    "Kuwait": "Asia",
    "Kyrgyz Republic": "Asia",
    "Kyrgyzstan": "Asia",
    "Lao PDR": "Asia",
    "Latvia": "Europe",
    "Lebanon": "Asia",
    "Lesotho": "Africa",
    "Liberia": "Africa",
    "Libya": "Africa",
    "Liechtenstein": "Europe",
    "Lithuania": "Europe",
    "Luxembourg": "Europe",
    "Madagascar": "Africa",
    "Malawi": "Africa",
    "Malaysia": "Asia",
    "Maldives": "Asia",
    "Mali": "Africa",
    "Malta": "Europe",
    "Marshall Islands": "Oceania",
    "Mauritania": "Africa",
    "Mauritius": "Africa",
    "Mexico": "North America",
    "Micronesia, Fed. Sts.": "Oceania",
    "Moldova": "Europe",
    "Monaco": "Europe",
    "Mongolia": "Asia",
    "Montenegro": "Europe",
    "Morocco": "Africa",
    "Mozambique": "Africa",
    "Myanmar": "Asia",
    "Namibia": "Africa",
    "Nauru": "Oceania",
    "Nepal": "Asia",
    "Netherlands": "Europe",
    "New Zealand": "Oceania",
    "Nicaragua": "North America",
    "Niger": "Africa",
    "Nigeria": "Africa",
    "North Macedonia": "Europe",
    "Norway": "Europe",
    "Oman": "Asia",
    "Pakistan": "Asia",
    "Palau": "Oceania",
    "Panama": "North America",
    "Papua New Guinea": "Oceania",
    "Paraguay": "South America",
    "Peru": "South America",
    "Philippines": "Asia",
    "Poland": "Europe",
    "Portugal": "Europe",
    "Qatar": "Asia",
    "Romania": "Europe",
    "Russian Federation": "Europe",
    "Rwanda": "Africa",
    "Samoa": "Oceania",
    "San Marino": "Europe",
    "Sao Tome and Principe": "Africa",
    "Saudi Arabia": "Asia",
    "Senegal": "Africa",
    "Serbia": "Europe",
    "Seychelles": "Africa",
    "Sierra Leone": "Africa",
    "Singapore": "Asia",
    "Slovak Republic": "Europe",
    "Slovenia": "Europe",
    "Solomon Islands": "Oceania",
    "Somalia": "Africa",
    "South Africa": "Africa",
    "South Sudan": "Africa",
    "Spain": "Europe",
    "Sri Lanka": "Asia",
    "St. Kitts and Nevis": "North America",
    "St. Lucia": "North America",
    "St. Vincent and the Grenadines": "North America",
    "Sudan": "Africa",
    "Suriname": "South America",
    "Sweden": "Europe",
    "Switzerland": "Europe",
    "Syrian Arab Republic": "Asia",
    "Tajikistan": "Asia",
    "Tanzania": "Africa",
    "Thailand": "Asia",
    "Timor-Leste": "Asia",
    "Togo": "Africa",
    "Tonga": "Oceania",
    "Trinidad and Tobago": "North America",
    "Tunisia": "Africa",
    "Turkey": "Asia",
    "Turkmenistan": "Asia",
    "Tuvalu": "Oceania",
    "Uganda": "Africa",
    "Ukraine": "Europe",
    "United Arab Emirates": "Asia",
    "United Kingdom": "Europe",
    "United States": "North America",
    "Uruguay": "South America",
    "Uzbekistan": "Asia",
    "Vanuatu": "Oceania",
    "Venezuela": "South America",
    "Venezuela, RB": "South America",
    "Vietnam": "Asia",
    "Viet Nam": "Asia",
    "West Bank and Gaza": "Asia",
    "Yemen": "Asia",
    "Yemen, Rep.": "Asia",
    "Zambia": "Africa",
    "Zimbabwe": "Africa"
}

# Crear la nueva columna "Continente"
data['Continente'] = data['country.value'].map(country_to_continent)

# Ordenar los países por continente y luego por nombre
sorted_data = data.sort_values(by=['Continente', 'country.value'])

# Guardar el archivo CSV resultante
output_path = 'world_bank_continentes.csv'
sorted_data.to_csv(output_path, index=False)

print(f"Archivo generado: {output_path}")

### 2.3 Filtrar CSV: Separación por ingresos

In [ ]:
# Lista de valores a filtrar en la última columna
valores_deseados = [
    "High income",
    "Low & middle income",
    "Low income",
    "Middle income",
    "Upper middle income"
]

# Filtrar las filas que contienen estos valores en la última columna
ultima_columna = world_bank_anemia_limpio.columns[-1]  # Nombre de la última columna
world_bank_anemia_filtrado = world_bank_anemia_limpio[world_bank_anemia_limpio[ultima_columna].str.strip().isin(valores_deseados)]

### 2.3.1 Mejoramos el nombre de las columnas y valor de filas:

In [ ]:
# Cambiar los nombres de las columnas
world_bank_anemia_filtrado = world_bank_anemia_filtrado.rename(columns={
    'date': 'year',
    'value': 'prevalencia (%)',
    'country.value': 'nivel de ingresos'
})

# Modificar los valores de la columna "nivel de ingresos"
world_bank_anemia_filtrado['nivel de ingresos'] = world_bank_anemia_filtrado['nivel de ingresos'].replace({
    'High income': 'Ingresos altos',
    'Low & middle income': 'Ingresos bajos y medios',
    'Low income': 'Bajos ingresos',
    'Middle income': 'Ingreso medio',
    'Upper middle income': 'Ingreso medio alto',
    'Lower middle income': 'Ingreso medio bajo'
})

# Guardar el nuevo archivo CSV con los nombres de columnas cambiados
archivo_csv_nuevo = "world_bank_anemia_ingresos_listo.csv"
world_bank_anemia_filtrado.to_csv(archivo_csv_nuevo, index=False)

print(f"Archivo con las columnas renombradas guardado como {archivo_csv_nuevo}")

### 2.4 Filtrar CSV: A nivel mundial

In [ ]:
# Lista de valores a filtrar en la última columna
valores_deseados = [
    "World"
]

# Filtrar las filas que contienen estos valores en la última columna
ultima_columna = world_bank_anemia_limpio.columns[-1]  # Nombre de la última columna
df_filtrado = world_bank_anemia_limpio[world_bank_anemia_limpio[ultima_columna].str.strip().isin(valores_deseados)]

# Guardar en un nuevo archivo CSV
nuevo_csv = "world_bank_anemia_mundial.csv"
df_filtrado.to_csv(nuevo_csv, index=False)

print(f"Datos filtrados guardados en: {nuevo_csv}")

### 2.4.1 Mejoramos el nombre de las columnas y valor de filas:

In [ ]:
# Cargar el archivo CSV
archivo_csv = "world_bank_anemia_mundial.csv"  # Asegúrate de que el archivo esté en la misma carpeta o proporciona la ruta completa
df = pd.read_csv(archivo_csv)

# Cambiar los nombres de las columnas
df = df.rename(columns={
    'date': 'year',
    'value': 'prevalencia (%)',
    'country.value': 'nivel geográfico'
})

# Modificar los valores de la columna "nivel de ingresos"
df['nivel geográfico'] = df['nivel geográfico'].replace({
    'World': 'Mundial'
})

# Guardar el nuevo archivo CSV con los nombres de columnas cambiados
archivo_csv_nuevo = "world_bank_anemia_mundial_listo.csv"
df.to_csv(archivo_csv_nuevo, index=False)

print(f"Archivo con las columnas renombradas guardado como {archivo_csv_nuevo}")

In [ ]:
# Lista de valores a filtrar en la última columna
valores_deseados = [
    "World"
]

# Filtrar las filas que contienen estos valores en la última columna
ultima_columna = world_bank_anemia_limpio.columns[-1]  # Nombre de la última columna
df_filtrado = world_bank_anemia_limpio[world_bank_anemia_limpio[ultima_columna].str.strip().isin(valores_deseados)]

# Guardar en un nuevo archivo CSV
nuevo_csv = "world_bank_anemia_mundial.csv"
df_filtrado.to_csv(nuevo_csv, index=False)

print(f"Datos filtrados guardados en: {nuevo_csv}")

# API: Demographic Health Survey (PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON CUALQUIER/LEVE/MODERADO/SEVERO NIVEL DE ANEMIA)

In [ ]:
import pandas as pd

# Cargar el archivo proporcionado por el usuario
file_path = 'df_any_anemia.csv'
any = pd.read_csv(file_path)

file_path = 'df_mld_anemia.csv'
mild = pd.read_csv(file_path)

file_path = 'df_mld_anemia.csv'
mod = pd.read_csv(file_path)

file_path = 'df_mld_anemia.csv'
sev = pd.read_csv(file_path)

## 1. Limpieza de Datos

### 1.1 Eliminación de columnas innecesarias

In [ ]:
# Lista de columnas a eliminar
columns_to_drop = ['DataId', 'SurveyId', 'Indicator', 'IsPreferred', 'SDRID', 'Precision', 'RegionId', 'SurveyType',
'IndicatorId', 'CharacteristicOrder', 'CharacteristicLabel',  'ByVariableLabel', 'CIHigh', 'IsTotal', 'ByVariableId',
                   'IndicatorOrder', 'DHS_CountryCode',  'CILow', 'LevelRank', 'CharacteristicId', 'CharacteristicCategory'
                 , 'IndicatorType',
                   'DenominatorUnweighted','DenominatorWeighted', "SurveyYearLabel", "Value"
]

# Eliminar las columnas no deseadas
df_cleaned0 = any.drop(columns=columns_to_drop)
df_cleaned1 = mild.drop(columns=columns_to_drop)
df_cleaned2 = mod.drop(columns=columns_to_drop)
df_cleaned3 = sev.drop(columns=columns_to_drop)

## 1.2 Mejorar nombres de columnas

In [ ]:
# Crear un nuevo dataframe con las columnas especificadas
df_combined = pd.DataFrame({
    'Year': any['SurveyYear'],  # SurveyYear de 'any'
    'Pais': any['CountryName'],  # CountryName de 'any'

    'Valor Cualquier': any['Value'],  # Value de 'any' renombrado
    '# Encuestas (any, sin ponderar)': any['DenominatorUnweighted'],  # DenominatorUnweighted de 'any'
    '# Encuestas (any, ponderadas)': any['DenominatorWeighted'],  # DenominatorWeighted de 'any'

    'Valor Leve': mild['Value'],  # Value de 'mild' renombrado
    '# Encuestas (mild, sin ponderar)': mild['DenominatorUnweighted'],  # DenominatorUnweighted de 'mild'
    '# Encuestas (mild, ponderadas)': mild['DenominatorWeighted'],  # DenominatorWeighted de 'mild'

    'Valor Moderado': mod['Value'],  # Value de 'mod' renombrado
    '# Encuestas (mod, sin ponderar)': mod['DenominatorUnweighted'],  # DenominatorUnweighted de 'mod'
    '# Encuestas (mod, ponderadas)': mod['DenominatorWeighted'],  # DenominatorWeighted de 'mod'

    'Valor Severo': sev['Value'],  # Value de 'sev' renombrado
    '# Encuestas (sev, sin ponderar)': sev['DenominatorUnweighted'],  # DenominatorUnweighted de 'sev'
    '# Encuestas (sev, ponderadas)': sev['DenominatorWeighted']  # DenominatorWeighted de 'sev'
})



# Guardar el dataframe combinado como un nuevo archivo CSV
output_file_combined = 'dhs_anemia_final.csv'
df_combined.to_csv(output_file_combined, index=False)

# Imprimir la ruta del archivo guardado
print(f"Archivo guardado en: {output_file_combined}")

# API KAGGLE (FACTORES QUE PODRIAN ESTAR INFLUENCIANDO EL NIVEL DE ANEMIA EN NIÑOS DE 0-59 MESES) - Caso: Nigeria

## Descripción del caso

En este estudio, se recopilaron datos transversales de las Encuestas demográficas y de salud de Nigeria (NDHS) de 2018 para responder a preguntas de investigación sobre el efecto de la edad de las madres y otros factores socioeconómicos en el nivel de anemia de los niños de 0 a 59 meses en Nigeria. Las DHS son encuestas transversales de hogares representativas a nivel nacional que generalmente se realizan cada 5 años. Los datos de esta encuesta consideraron los 36 estados de Nigeria, así como el Territorio de la Capital Federal (FCT). La población objetivo de este estudio son los niños de 0 a 59 meses y las madres de 15 a 49 años. En esta encuesta, el ingreso del hogar se midió utilizando el índice de riqueza, la edad actual en grupos de 5 años se produce agrupando la edad actual en años completados, tipo de lugar de residencia donde el encuestado fue entrevistado como urbano o rural, la categorización se creó en función de si el número de punto de muestra o conglomerado se define como urbano o rural, el nivel más alto de educación alcanzado es una variable estandarizada que proporciona el nivel de educación en las siguientes categorías: Sin educación, Educación primaria, secundaria y superior, el número total de nacimientos en los últimos cinco años se define como todos los nacimientos en los meses 0 a 59 anteriores al mes de la entrevista, donde el mes 0 es el mes de la entrevista, la edad del encuestado en el primer nacimiento se calcula utilizando el CMC de la fecha de nacimiento del encuestado.

Después de la depuración de los datos, se utilizó el método Chi cuadrado para probar las hipótesis sobre la posible relación que existe entre ciertos factores socioeconómicos y los niveles de anemia en niños de 0 a 59 meses. El nivel de anemia fue la variable predictora y las variables explicativas son la edad de la madre, el nivel de educación, el índice de riqueza, el nacimiento en los últimos cinco años, el uso de mosquiteros, etc.

# Diccionario de datos

```Python
Type of place of residence
0: Rural
1: Urban


Highest educational level
0: Higher
1: No education
2: Primary
3: Secondary


Wealth index combined
0: Middle
1: Poorer
2: Poorest
3: Richer
4: Richest


Anemia level
0: Mild
1: Moderate
2: Not anemic
3: Severe


Have mosquito bed net for sleeping (from household questionnaire)
0: No
1: Yes


Smokes cigarettes
0: No
1: Yes


Current marital status
0: Divorced
1: Living with partner
2: Married
3: Never in union
4: No longer living together/separated
5: Widowed


Currently residing with husband/partner
0: Living with her
1: Staying elsewhere


When child put to breast
0: 102.0
1: 103.0
2: 104.0
... (continuando con valores similares)
38: Days: 1
39: Hours: 1
40: Immediately


Had fever in last two weeks
0: Don't know
1: No
2: Yes


Taking iron pills, sprinkles or syrup
0: Don't know
1: No
2: Yes
```





## 1. Implementacion de funciones para limpiar, ordenar y transformar los datos.

In [ ]:
import pandas as pd

# Cargar el archivo proporcionado por el usuario
file_path = 'children anemia.csv'
data = pd.read_csv(file_path)

# Mostrar una vista previa de los datos para analizar la estructura
data.head(), data.info()

In [ ]:
# Funciones para limpiar, ordenar y transformar los datos
def limpiar_datos(dataframe):
    """
    Limpia los datos eliminando columnas duplicadas, renombrando columnas y gestionando valores faltantes.
    """
    # Eliminar columnas duplicadas o irrelevantes
    columnas_a_eliminar = ['Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)',
                           'Anemia level.1']
    dataframe = dataframe.drop(columns=columnas_a_eliminar, errors='ignore')

    # Renombrar columnas para mayor claridad
    dataframe = dataframe.rename(columns={
        'Age in 5-year groups': 'Age_Group',
        'Type of place of residence': 'Residence_Type',
        'Highest educational level': 'Education_Level',
        'Wealth index combined': 'Wealth_Index',
        'Births in last five years': 'Births_Last_5_Years',
        'Age of respondent at 1st birth': 'Age_First_Birth',
        'Anemia level': 'Anemia_Level',
        'Have mosquito bed net for sleeping (from household questionnaire)': 'Mosquito_Net',
        'Smokes cigarettes': 'Smokes',
        'Current marital status': 'Marital_Status',
        'Currently residing with husband/partner': 'Residing_With_Partner',
        'When child put to breast': 'Breastfeeding_Timing',
        'Had fever in last two weeks': 'Fever_Last_2_Weeks',
        'Hemoglobin level adjusted for altitude (g/dl - 1 decimal)': 'Hemoglobin_Level',
        'Taking iron pills, sprinkles or syrup': 'Iron_Supplements'
    })

    # Manejo de valores faltantes
    dataframe['Anemia_Level'] = dataframe['Anemia_Level'].fillna('Unknown')  # Llenar valores faltantes de anemia con "Unknown"
    dataframe = dataframe.dropna(subset=['Hemoglobin_Level', 'Education_Level'])  # Eliminar filas con valores críticos faltantes

    return dataframe


def transformar_datos(dataframe):
    """
    Transforma los datos categóricos a variables numéricas y estandariza las columnas.
    """
    # Convertir categorías a valores numéricos
    categoricas_a_codificar = ['Residence_Type', 'Education_Level', 'Wealth_Index', 'Anemia_Level',
                               'Mosquito_Net', 'Smokes', 'Marital_Status', 'Residing_With_Partner',
                               'Breastfeeding_Timing', 'Fever_Last_2_Weeks', 'Iron_Supplements']

    for columna in categoricas_a_codificar:
        dataframe[columna] = dataframe[columna].astype('category').cat.codes

    return dataframe


# Cargar el archivo CSV
file_path = 'children anemia.csv'  # Reemplaza con la ruta del archivo en tu sistema
data = pd.read_csv(file_path)

# Aplicar funciones al dataset
datos_limpios = limpiar_datos(data)
datos_transformados = transformar_datos(datos_limpios)

# Mostrar las primeras filas del DataFrame transformado
print(datos_transformados.head())

# Guardar los datos transformados en un archivo CSV
datos_transformados.to_csv('datos_limpios_transformados.csv', index=False)
print("Datos guardados como 'datos_limpios_transformados.csv'.")

## 2. Calcular métricas como media, mediana, moda, y otras estadísticas relevantes

In [ ]:
def calcular_metricas_avanzadas(dataframe):
    """
    Calcula métricas descriptivas avanzadas como percentiles, asimetría, curtosis y más.
    """
    # Seleccionar columnas numéricas
    columnas_numericas = dataframe.select_dtypes(include=['int64', 'float64'])

    # Crear un diccionario para almacenar las métricas
    estadisticas = {}

    for columna in columnas_numericas.columns:
        estadisticas[columna] = {
            'Media': columnas_numericas[columna].mean(),
            'Mediana': columnas_numericas[columna].median(),
            'Moda': columnas_numericas[columna].mode().iloc[0] if not columnas_numericas[columna].mode().empty else None,
            'Desviación Estándar': columnas_numericas[columna].std(),
            'Mínimo': columnas_numericas[columna].min(),
            'Máximo': columnas_numericas[columna].max(),
            'Rango': columnas_numericas[columna].max() - columnas_numericas[columna].min(),
            'Percentil 25': columnas_numericas[columna].quantile(0.25),
            'Percentil 75': columnas_numericas[columna].quantile(0.75),
            'Asimetría': columnas_numericas[columna].skew(),
            'Curtosis': columnas_numericas[columna].kurt(),
            'Rango Intercuartílico (IQR)': columnas_numericas[columna].quantile(0.75) - columnas_numericas[columna].quantile(0.25),
            'Coeficiente de Variación (CV)': columnas_numericas[columna].std() / columnas_numericas[columna].mean(),
        }

    # Convertir a un DataFrame para mejor visualización
    estadisticas_df = pd.DataFrame(estadisticas).transpose()
    return estadisticas_df


# Calcular métricas avanzadas
metricas_avanzadas = calcular_metricas_avanzadas(datos_transformados)

# Mostrar las métricas avanzadas
print(metricas_avanzadas)


## 3. Estructuras de datos (listas, pilas, colas) que faciliten el análisis

### 3.1 Lista

Las listas son ideales para almacenar datos tabulares o registros específicos que se necesitan procesar en secuencia

In [ ]:
# Extraer datos relevantes y almacenarlos en una lista
lista_anemia = datos_transformados[['Age_Group', 'Anemia_Level', 'Hemoglobin_Level']].values.tolist()

# Ejemplo de acceso a los datos
print("Ejemplo de registros en lista:")
print(lista_anemia[:5])  # Imprime los primeros 5 registros


### 3.2 Pila (Stack)

Las pilas siguen el principio LIFO (Last In, First Out) y son útiles si los datos se necesitan procesar en orden inverso.

In [ ]:
class Pila:
    def __init__(self):
        self.stack = []

    def push(self, item):
        self.stack.append(item)

    def pop(self):
        return self.stack.pop() if not self.is_empty() else None

    def peek(self):
        return self.stack[-1] if not self.is_empty() else None

    def is_empty(self):
        return len(self.stack) == 0

# Crear una pila con los datos relevantes
pila_anemia = Pila()
for _, row in datos_transformados.iterrows():
    pila_anemia.push({'Age_Group': row['Age_Group'], 'Anemia_Level': row['Anemia_Level']})

# Ejemplo de uso de la pila
print("Dato extraído de la pila:", pila_anemia.pop())


### 3.3 Cola (Queue)

Las colas siguen el principio FIFO (First In, First Out) y son útiles si los datos se deben procesar en el mismo orden en que se almacenaron.

In [ ]:
from collections import deque

# Crear una cola con los datos relevantes
cola_anemia = deque()
for _, row in datos_transformados.iterrows():
    cola_anemia.append({'Age_Group': row['Age_Group'], 'Anemia_Level': row['Anemia_Level']})

# Ejemplo de uso de la cola
print("Dato extraído de la cola:", cola_anemia.popleft())


## API: Global Health Observatory - Prevalencia de anemia infantil (% de anemia infantil entre los 6-59 meses)

## 1. Analisis de datos inicial

In [ ]:
import requests
import pandas as pd

# URL de la API para la prevalencia de anemia (sustituir con el código correcto)
indicator_code = "NUTRITION_ANAEMIA_CHILDREN_PREV"  # Cambia a la API para prevalencia de anemia
url = f"https://ghoapi.azureedge.net/api/{indicator_code}"

# Realizar la solicitud
response = requests.get(url)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Extraer los datos en formato JSON

    # Convertir los datos a un DataFrame
    df = pd.json_normalize(data['value'])
    # Mostrar las primeras filas
    print(df.head(48))

else:
    print(f"Error al acceder a la API: {response.status_code}")

## **2. Limpieza de datos**


In [ ]:
print(df.isnull().sum())

### **2.1 Eliminación de filas y columnas sin datos**

*   Elemento de la lista
*   Elemento de la lista

In [ ]:
# Eliminar filas con valores nulos
df_cleaned = df.dropna()

# Eliminar columnas con valores nulos
df_cleaned = df.dropna(axis=1)

In [ ]:
print("Columnas después de la limpieza:")
print(df_cleaned.columns)

### **2.2 Imputación de Datos**

In [ ]:
# Seleccionar columnas específicas
# Asegúrate de que los nombres de las columnas coincidan exactamente con los del DataFrame
columnas_deseadas = ['SpatialDimType', 'SpatialDim', 'Dim1Type', 'TimeDim','NumericValue', 'Low', 'High']
# Elimina espacios extra en ' Dim1Type' y 'TimeDim '

# Verificar si las columnas existen antes de seleccionarlas
columnas_existentes = df_cleaned.columns
columnas_deseadas = [col for col in columnas_deseadas if col in columnas_existentes]

df_seleccion = df_cleaned[columnas_deseadas]

print(df_seleccion.head(30))

## 3. Organización de datos

In [ ]:
import pandas as pd

# Asegurando que la columna tiene valores correctos
# Aquí supongo que df es tu DataFrame
df['SpatialDim'] = df['SpatialDim'].astype(str)

# Filtrar los valores no numéricos y encontrar la moda
moda_no_numerica = df[~df['SpatialDim'].str.isnumeric()]['SpatialDim'].mode()[0]

# Reemplazar los valores numéricos por la moda
df['SpatialDim'] = df['SpatialDim'].apply(lambda x: moda_no_numerica if x.isnumeric() else x)

# Mostrar los primeros registros para verificar
print(df_seleccion.head(30))

### **3.1 Obtener años únicos por país en un DataFrame ordenado**

In [ ]:
# Agrupar los años únicos por cada valor de SpatialDim
years_by_spatialdim = df[df['SpatialDimType'] == 'COUNTRY'].groupby('SpatialDim')['TimeDim'].unique()

# Convertir a un DataFrame para una mejor presentación
result = years_by_spatialdim.apply(sorted).reset_index(name='Years')

# Mostrar el resultado
print(result)

#### **3.2 COMPARAR AÑOS CON MAYOR Y MENOR PROMEDIO USANDO LISTAS**

In [ ]:
# Lista para almacenar promedios anuales
year_averages = []

# Calcular promedio por año
for year in df['TimeDim'].unique():
    values = list(df[df['TimeDim'] == year]['NumericValue'])
    avg = sum(values) / len(values)
    year_averages.append((year, avg))

# Ordenar por promedio
year_averages.sort(key=lambda x: x[1])

# Mostrar resultados
print(f"Año con menor promedio: {year_averages[0][0]} - Promedio: {year_averages[0][1]:.2f}")
print(f"Año con mayor promedio: {year_averages[-1][0]} - Promedio: {year_averages[-1][1]:.2f}")

#### **4.1 PREVALENCIA DE ANEMIA POR CADA AÑO EN LOS DIFERENTES PAISES**

# Filtrar registros donde SpatialDimType sea "COUNTRY"
df_filtered = df[df['SpatialDimType'] == 'COUNTRY']

# Calcular el promedio de prevalencia de anemia por país y año
average_anemia = df_filtered.groupby(['SpatialDim', 'TimeDim'])['NumericValue'].mean().reset_index()

# Renombrar las columnas para mayor claridad
average_anemia.columns = ['Country', 'Year', 'AveragePrevalence']

# Mostrar los primeros registros del resultado
print(average_anemia.head(30))



### 4.1.1 Descargar el archivo en un csv

In [ ]:
from google.colab import files
import pandas as pd

# ... (Your existing code to generate average_anemia DataFrame) ...

# Save the DataFrame to a CSV file
average_anemia.to_csv('PREVALENCIA_DE_ANEMIA_POR_CADA_AÑO_EN_LOS_DIFERENTES_PAISES.csv', index=False)

# Download the CSV file
files.download('PREVALENCIA_DE_ANEMIA_POR_CADA_AÑO_EN_LOS_DIFERENTES_PAISES.csv')

#### **4.2 PROMEDIO DE PREVALENCIA POR PAÍS USANDO LISTAS**

In [ ]:
 # Lista para almacenar los promedios
country_averages = []

# Agrupar por país y calcular el promedio manualmente
for country in df['SpatialDim'].unique():
    if df[df['SpatialDim'] == country]['SpatialDimType'].iloc[0] == 'COUNTRY':
        prevalences = list(df[df['SpatialDim'] == country]['NumericValue'])
        avg_prevalence = sum(prevalences) / len(prevalences)
        country_averages.append((country, avg_prevalence))

# Mostrar los resultados
for country, avg in country_averages:
    print(f"{country}: {avg:.2f}")

### 4.2.1 Descargar el archivo en un csv

In [ ]:
# Crear un DataFrame a partir de los resultados
results_df = pd.DataFrame(country_averages, columns=['Country', 'AveragePrevalence'])

# Guardar el DataFrame como archivo CSV
results_df.to_csv('PROMEDIO_DE_PREVALENCIA_POR_PAÍS_USANDO_LISTAS_gho.csv', index=False)

# Descargar el archivo a tu ordenador
from google.colab import files
files.download('PROMEDIO_DE_PREVALENCIA_POR_PAÍS_USANDO_LISTAS_gho.csv')

#### **4.3 RANGO DE PREVALENCIA POR AÑO USANDO LISTAS**

In [ ]:
import pandas as pd

# Crear una pila para almacenar rangos por año
ranges_stack = []

# Agrupar por año
for year in df['TimeDim'].unique():
    values = list(df[df['TimeDim'] == year]['NumericValue'])
    max_val = max(values)
    min_val = min(values)
    year_range = max_val - min_val
    ranges_stack.append((year, min_val, max_val, year_range))

# Crear un DataFrame con las columnas correspondientes
ranges_df = pd.DataFrame(ranges_stack, columns=['Año', 'Min', 'Max', 'Rango'])

# Mostrar el DataFrame
print(ranges_df)

###### **4.3.1 DESCARGAR EN FORMATO CSV**

In [ ]:
# Guardar el DataFrame como archivo CSV
ranges_df.to_csv('RANGO_DE_PREVALENCIA_POR_AÑO_USANDO_LISTAS_gho.csv', index=False)

# Descargar el archivo a tu ordenador
from google.colab import files
files.download('RANGO_DE_PREVALENCIA_POR_AÑO_USANDO_LISTAS_gho.csv')

#### **4.4 CALCULAR LA MEDIANA DE PREVALENCIA USANDO COLAS**

In [ ]:
# Calcular la mediana por año
medians_queue = deque()

# Agrupar por año
for year in df['TimeDim'].unique():
    values = sorted(list(df[df['TimeDim'] == year]['NumericValue']))
    n = len(values)
    if n % 2 == 0:  # Si hay un número par de valores
        median = (values[n // 2 - 1] + values[n // 2]) / 2
    else:  # Si hay un número impar de valores
        median = values[n // 2]
    medians_queue.append((year, median))

# Convertir los resultados a un DataFrame
medians_df = pd.DataFrame(medians_queue, columns=['Año', 'Mediana'])

# Mostrar el DataFrame
print(medians_df)

##### **4.4.1 GUARDAR EN FORMATO CSV**

In [ ]:
# Guardar el DataFrame como archivo CSV
medians_df.to_csv('LA_MEDIANA_DE_PREVALENCIA_USANDO_COLAS_gho.csv', index=False)

# Descargar el archivo a tu ordenador
from google.colab import files
files.download('LA_MEDIANA_DE_PREVALENCIA_USANDO_COLAS_gho.csv')

#### **4.5 COMPARAR PAÍSES CON MÁS ESTABILIDAD**

In [ ]:
import pandas as pd

# Lista para almacenar rangos por país
country_ranges = []

# Calcular rango por país
for country in df['SpatialDim'].unique():
    if df[df['SpatialDim'] == country]['SpatialDimType'].iloc[0] == 'COUNTRY':
        values = list(df[df['SpatialDim'] == country]['NumericValue'])
        country_range = max(values) - min(values)
        country_ranges.append((country, country_range))

# Ordenar por rango
country_ranges.sort(key=lambda x: x[1])

# Convertir a un DataFrame
ranges_df = pd.DataFrame(country_ranges, columns=['País', 'Rango'])

# Mostrar el DataFrame
print(ranges_df)  # Mostrar los 60 países más estables

#### **4.5.1 DESCARGAR EN FORMATO CSV**

In [ ]:
# Guardar el DataFrame como archivo CSV
ranges_df.to_csv('COMPARAR_PAÍSES_CON_MÁS_ESTABILIDAD_gho.csv', index=False)

# Descargar el archivo a tu ordenador
from google.colab import files
files.download('COMPARAR_PAÍSES_CON_MÁS_ESTABILIDAD_gho.csv')

## 5. Visualización de datos

### 5.1 Paquetes requeridos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import plotly.express as px #Para hacer graficos interactivos, no funcionaba en mapamundi
from ipywidgets import interact
from IPython.display import display, clear_output
from matplotlib.patches import Wedge
from pandas.plotting import table
from plotly.subplots import make_subplots #Para poder poner subgraficos categorizados
import plotly.graph_objects as go
import random
import seaborn as sns

#Para el mapamundi
#!pip install folium
#import folium

#!pip install geopandas
#!pip install geopy

#No se podía poner el filtro dinámico, gpt comentó instalar --> #jupyter nbextension enable --py widgetsnbextension
#generaba un error porque la versión de Jupyter no lo soportaba.
#Por lo que se actualizo con este código --> pip install --upgrade jupyter jupyterlab ipywidgets

### 5.2 Anemia infantil a través del tiempo (2000-2019)

#### 5.2.1 Anemia infantil a nivel mundial

In [ ]:
#Fuente: World Bank

# Cargar datos
data_historico = pd.read_csv(r"world_bank_anemia_mundial_listo.csv")

# Crear el gráfico
fig = px.line(
    data_historico,
    x="year",
    y="prevalencia (%)",
    markers=True,
    title="Prevalencia de anemia (en %), 2000-2019",
    labels={"year": "Año", "prevalencia (%)": "Prevalencia (%)"}
)

# Personalizar diseño
fig.update_traces(
    hovertemplate="Prevalencia: %{y:.1f}%",
    line=dict(color="blue"),
    marker=dict(size=8)
)

fig.update_layout(
    yaxis=dict(title="Prevalencia (%)", range=[10, 50]),
    xaxis=dict(
        tickangle=-90,  # Rotar las etiquetas de los años
        tickmode='array',  # Mostrar todos los años
        tickvals=data_historico["year"],  # Asegura que todos los años se muestren
        title=""  # Eliminar el título del eje X
    ),
    title=dict(font=dict(size=18)),
    template="simple_white"
)

# Mostrar el gráfico
fig.show()

#### 5.2.2 Prevalencia de anemia infantil según nivel de ingreso

In [ ]:
#Fuente: World Bank

# Cargar datos
data_nivelingresos = pd.read_csv(r"world_bank_anemia_ingresos_listo.csv")

# Asegurarse de que los datos de 'year' sean numéricos
data_nivelingresos['year'] = pd.to_numeric(data_nivelingresos['year'], errors='coerce')
data_nivelingresos = data_nivelingresos.dropna(subset=['year', 'prevalencia (%)'])  # Eliminar filas con datos faltantes
data_nivelingresos['year'] = data_nivelingresos['year'].astype(int)

# Crear el gráfico interactivo con Plotly
fig = px.line(data_nivelingresos, x='year', y='prevalencia (%)', color='nivel de ingresos',
              markers=True, title='Prevalencia histórica de anemia por nivel de ingresos',
              labels={'prevalencia (%)': 'Prevalencia (%)', 'year': 'Año'})

# Eliminar la leyenda
fig.update_layout(showlegend=False)

# Añadir el nombre del nivel de ingresos cerca del último punto disponible y alineado verticalmente
y_offset = 0.1  # Ajuste de distancia entre los nombres de los niveles
for i, level in enumerate(data_nivelingresos['nivel de ingresos'].unique()):
    level_data = data_nivelingresos[data_nivelingresos['nivel de ingresos'] == level]
    last_row = level_data[level_data['year'] == level_data['year'].max()]
    if not last_row.empty:
        last_year = last_row['year'].values[0]
        last_value = last_row['prevalencia (%)'].values[0]
        
        # Añadir la anotación en la posición deseada
        fig.add_annotation(
            x=last_year + 0.4,  # Posición horizontal cerca del último punto
            y=last_value + (y_offset * i),  # Posición vertical con ajuste para separar los nombres
            text=level,  # Texto con el nombre del nivel de ingresos
            showarrow=False,  # Sin flecha
            font=dict(size=10),  # Tamaño de la fuente
            xanchor='left',  # Alineación del texto a la izquierda
            align='left',  # Alineación del texto a la izquierda
        )

# Mejorar el diseño del gráfico
fig.update_traces(mode='lines+markers', hovertemplate='Prevalencia: %{y}')  # Solo mostrar prevalencia en el tooltip
fig.update_layout(
    xaxis_title='Año',
    yaxis_title='Prevalencia (%)',
    xaxis=dict(
        title=None,  # Quitar el título del eje X
        tickmode='array', 
        tickvals=sorted(
            data_nivelingresos['year'].unique()),
        showline=True,
        linecolor='black',
        ticks='outside',  # Mostrar marcas de graduación principales hacia el exterior
        tickwidth=1,  # Grosor de las marcas de graduación
    ),
    yaxis=dict(
            showline=True,  # Mostrar la línea del eje Y
            linewidth=1,  # Definir el grosor de la línea
            linecolor='black'  # Definir el color de la línea
    ),
    xaxis_tickangle=-90,  # Girar el eje de los años en sentido opuesto
    plot_bgcolor='white',
    font=dict(size=12),
    title_x=0.5,  # Centrar el título
)

# Mostrar el gráfico
fig.show()

#### 5.3.3 Prevalencia de anemia infantil por país (Multiselección) 

In [ ]:
# Fuente: World Bank

# Cargar los datos
data_historico_pais_est = pd.read_csv(r"world_bank_anemia_paises_listo.csv")
data_historico_pais_est['year'] = pd.to_numeric(data_historico_pais_est['year'], errors='coerce')
data_historico_pais_est['year'] = data_historico_pais_est['year'].astype(int)

# Obtener la lista de países únicos
countries = sorted(data_historico_pais_est['pais'].unique())

# Asignar un color único a cada país
def assign_colors(countries):
    colors = {}
    for country in countries:
        # Asignamos un color aleatorio a cada país
        colors[country] = f'rgba({random.randint(0,255)},{random.randint(0,255)},{random.randint(0,255)}, 0.8)'
    return colors

colors = assign_colors(countries)

# Función para completar los años faltantes y hacer líneas continuas
def completar_anios(data, country):
    # Filtrar datos del país
    country_data = data[data['pais'] == country].copy()

    # Generar el rango completo de años
    all_years = pd.DataFrame({'year': range(country_data['year'].min(), country_data['year'].max() + 1)})

    # Unir con los datos originales y llenar los valores faltantes mediante interpolación
    completed_data = pd.merge(all_years, country_data, on='year', how='left')
    completed_data['prevalencia (%)'] = completed_data['prevalencia (%)'].interpolate()

    # Añadir el nombre del país
    completed_data['pais'] = country
    return completed_data

# Función para graficar prevalencias históricas basadas en los países seleccionados
def plot_selected_countries_plotly(countries_selected):
    if not countries_selected:
        with output_area:
            clear_output(wait=True)
            print("Por favor selecciona al menos un país.")
        return

    # Crear una figura
    fig = go.Figure()

    for country in countries_selected:
        # Completar los años faltantes
        country_data = completar_anios(data_historico_pais_est, country)

        # Dividir los datos en tres segmentos
        before_2020 = country_data[country_data['year'] < 2020]
        between_2020_2030 = country_data[(country_data['year'] >= 2020) & (country_data['year'] <= 2030)]
        after_2030 = country_data[country_data['year'] > 2030]

        # Obtener el color para el país
        country_color = colors[country]

        # Añadir el segmento antes de 2020 (línea continua)
        fig.add_trace(
            go.Scatter(
                x=before_2020['year'],
                y=before_2020['prevalencia (%)'],
                mode='lines+markers',
                name=country,
                hovertemplate="Prevalencia: %{y:.2f}<extra></extra>",  # Personalizar el tooltip sin el símbolo '%'
                line=dict(color=country_color)  # Usamos el color del país
            )
        )

        # Añadir el segmento entre 2020 y 2030 (línea punteada)
        fig.add_trace(
            go.Scatter(
                x=between_2020_2030['year'],
                y=between_2020_2030['prevalencia (%)'],
                mode='lines+markers',
                name=country,
                hovertemplate="Prevalencia: %{y:.2f}<extra></extra>",  # Personalizar el tooltip sin el símbolo '%'
                line=dict(dash='dot', color=country_color)  # Usamos el mismo color del país para la línea punteada
            )
        )

        # Añadir el segmento después de 2030 (línea continua)
        fig.add_trace(
            go.Scatter(
                x=after_2030['year'],
                y=after_2030['prevalencia (%)'],
                mode='lines+markers',
                name=country,
                hovertemplate="Prevalencia: %{y:.2f}<extra></extra>",  # Personalizar el tooltip sin el símbolo '%'
                line=dict(color=country_color)  # Usamos el color del país
            )
        )

        # Colocar el nombre del país ligeramente desplazado a la derecha de 2030
        year_2030_data = country_data[country_data['year'] == 2019]
        if not year_2030_data.empty:
            # Obtenemos el valor de prevalencia para 2030
            prev_2030 = year_2030_data['prevalencia (%)'].values[0]
            fig.add_annotation(
                x=2019 + 1.2,  # Desplazamos un poco a la derecha de 2030
                y=prev_2030,
                text=country,
                showarrow=False,
                font=dict(size=10, color='black'),
                xanchor='left',  # Alineación del texto a la izquierda
                align='left'  # Alineación del texto a la izquierda
            )

    # Ajustar el diseño del gráfico
    fig.update_layout(
        title={
            'text': 'Prevalencia histórica de anemia',
            'x': 0.5,  # Centrar el título
            'xanchor': 'center',  # Asegurar que el anclaje sea en el centro
        },
        xaxis=dict(
            title=None,  # Quitar el título del eje X
            tickangle=-90,
            showline=True,
            linecolor='black',
            ticks='outside',  # Mostrar marcas de graduación principales hacia el exterior
            tickwidth=1,  # Grosor de las marcas de graduación
            tickvals=list(
                range(
                    data_historico_pais_est['year'].min(), 
                    data_historico_pais_est['year'].max() + 1
                )
            )  # Asegurar que todos los años estén en el eje X
        ),
        yaxis=dict(
            showline=True,  # Mostrar la línea del eje Y
            linewidth=1,  # Definir el grosor de la línea
            linecolor='black'  # Definir el color de la línea
        ),
        showlegend=False,  # Quitar la leyenda
        yaxis_title='Prevalencia (%)',
        legend_title='Países',
        template='plotly_white',
        width=850  # Ampliar el ancho del gráfico,
    )

    # Mostrar el gráfico en el área de salida
    with output_area:
        clear_output(wait=True)
        fig.show()

# Función para manejar la actualización dinámica del gráfico
def update_plot(change=None):
    # Obtener los países seleccionados
    selected_countries = [cb.description for cb in checkboxes if cb.value]

    # Actualizar el gráfico
    plot_selected_countries_plotly(selected_countries)

# Crear checkboxes para cada país
checkboxes = [widgets.Checkbox(value=False, description=country) for country in countries]

# Agregar un evento a cada checkbox
for cb in checkboxes:
    cb.observe(update_plot, names='value')

# Crear un contenedor para los checkboxes con barra de desplazamiento
checkbox_box = widgets.VBox(checkboxes, layout=widgets.Layout(overflow='auto', height='300px', width='200px'))

# Crear un área de salida para el gráfico
output_area = widgets.Output()

# Mostrar los checkboxes y el área de salida
display(widgets.HBox([checkbox_box, output_area]))

#### 5.3.4 Clasificación de la importancia de la anemia infantil en la salud pública, por país

In [ ]:
# Fuente: World Bank

# Función para graficar la prevalencia de anemia de un país específico

data_ind_anemia = pd.read_csv(r"dhs_anemia_final.csv")
data_ind_anemia.drop(data_ind_anemia.columns[[3, 4, 5, 6, 7, 8, 9, 10]], axis=1, inplace=True)
data_ind_anemia.rename(
    columns={
    'Valor Cualquier': 'Valor\nReal',
    'Valor Severo' : 'Valor\nsevero',
    '# Encuestas (sev, sin ponderar)': '# Encuestas\n(sin ponderar)',
    '# Encuestas (sev, ponderadas)': '# Encuestas\n(ponderadas)'
    }, 
    inplace=True) 

# Función para crear un velocímetro estilizado
def plot_stylized_gauge(ax, value, country, min_val=0, max_val=100):
    levels = ["Baja", "Moderada", "Alta"]
    colors = ["#32CD32", "#FFD700", "#FF4D4D"]  # Verde -> Amarillo -> Rojo
    thresholds = [0, 20, 40, 100]

    start_angle = 180  # Inicio del semicírculo en 180° (sentido antihorario)
    end_angle = 0 # Fin del semicírculo

    for i, (level, color) in enumerate(zip(levels, colors)):

        # Calcular los ángulos para cada nivel en función de los umbrales
        start = start_angle - (start_angle - end_angle) * (thresholds[i + 1] - min_val) / (max_val - min_val)
        end = start_angle - (start_angle - end_angle) * (thresholds[i] - min_val) / (max_val - min_val)
        
        # Dibujar sección usando Wedge
        wedge = Wedge(center=(0, 0), r=1, theta1=start, theta2=end, facecolor=color, edgecolor="white")
        ax.add_patch(wedge)

        # Posicionar texto
        mid_angle = (start + end) / 2
        x_text = np.cos(np.radians(mid_angle)) * 0.7
        y_text = np.sin(np.radians(mid_angle)) * 0.7
        ax.text(x_text, y_text, level, ha="center", va="center", fontsize=14, color="white")

    # Dibujar la aguja (en sentido antihorario)
    value_angle = start_angle - (start_angle - end_angle) * (value - min_val) / (max_val - min_val)
    x_needle = np.cos(np.radians(value_angle)) * 0.9
    y_needle = np.sin(np.radians(value_angle)) * 0.9
    ax.plot([0, x_needle], [0, y_needle], color="black", linewidth=2)

    # Agregar valor al centro
    ax.text(0, -0.2, f"{value}", ha="center", va="center", fontsize=18, weight="bold")
    ax.text(0, -0.35, country, ha="center", va="center", fontsize=12, style="italic")

    # Ajustar límites y ocultar ejes
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-0.0025, 1.2)  # Mostrar solo el semicírculo superior
    ax.set_aspect('equal')  # Relación de aspecto 1:1 para un semicírculo perfecto
    ax.axis("off")

# Función para graficar la tabla de prevalencia 
def plot_paises(paises):
    # Filtrar los datos para el país seleccionado
    data_paises = data_ind_anemia[data_ind_anemia['Pais'] == paises]
    
    # Obtener el valor "severo" del año más reciente
    latest_year = data_paises['Year'].max()
    severe_value = data_paises[data_paises['Year'] == latest_year]['Valor\nReal'].values[0] 

    # Crear figura con dos subgráficos (uno para la tabla y otro para el velocímetro)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))  # Tamaño de la figura

    # Mostrar la tabla en el primer subgráfico
    ax1.axis('off')  # Apagar los ejes para la tabla
    
    # Crear la tabla usando ax.table
    table_data = data_paises.values
    table_columns = data_paises.columns
    table = ax1.table(cellText=table_data, colLabels=table_columns, loc='center', cellLoc='center')

    # Personalizar el estilo de la tabla
    table.auto_set_font_size(False)
    table.set_fontsize(12)

    # Ajustar las celdas
    for (i, j), cell in table.get_celld().items():
        cell.set_edgecolor('black')
        cell.set_linewidth(1.2)
        if i == 0:  # Encabezados
            cell.set_height(0.25)
            cell.set_facecolor('#4CAF50')
            cell.set_text_props(ha='center', va='center', color='white', weight='bold', fontsize=10)
        else:  # Filas de datos
            cell.set_facecolor('#f9f9f9')
            cell.set_text_props(color='black', fontsize=10)
        if j == 0:  # Primera columna
            cell.set_facecolor('#d3d3d3')
            cell.set_text_props(color='black', weight='bold')
            
    # Ajustar el ancho de las columnas basado en el texto
    col_widths = [max(len(str(cell)) for cell in data_paises[col].tolist() + [col]) for col in data_paises.columns]
    total_width = sum(col_widths)
    for i, width in enumerate(col_widths):
        table.auto_set_column_width(i)  # Ajustar automáticamente
        table.get_celld()[(0, i)].set_width(width / total_width * 2)  # Normalizar al tamaño del gráfico

    # Mostrar el velocímetro en el segundo subgráfico
    value = data_paises['Valor\nReal'].values[0]  # Suponiendo que 'Valor Cualquier' es el valor que usas
    plot_stylized_gauge(ax2, severe_value, paises)

    plt.tight_layout()
    plt.show()

# Crear el selector interactivo
lista_paises = data_ind_anemia['Pais'].unique()
interact(plot_paises, paises=lista_paises)